In [3]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset

In [2]:
# load model and processor
checkpoint = "openai_whisper-large/experiment_1/checkpoint-2000"
processor = WhisperProcessor.from_pretrained("openai/whisper-large")
model = WhisperForConditionalGeneration.from_pretrained(checkpoint)
forced_decoder_ids = processor.get_decoder_prompt_ids(language="hausa", task="transcribe")

# load streaming dataset and read first audio sample
#ds = load_dataset("mozilla-foundation/common_voice_9_0", "ha", split="test", streaming=True)
#ds = ds.cast_column("audio", Audio(sampling_rate=16_000))
cache_dir="/data/users/kashrest/asr-experiments"
data = load_dataset("google/fleurs", "ha_ng", split="test", cache_dir=cache_dir)

input_speech = next(iter(data))["audio"]

data_iter = iter(data)
transcriptions_val = []
for instance in data_iter:
    input_speech = instance["audio"]
    input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="pt").input_features

    # generate token ids
    predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
    # decode token ids to text
    transcription = processor.batch_decode(predicted_ids)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    transcriptions_val.append(transcription)

KeyboardInterrupt: 

In [ ]:
import json
with open("whisper-large-finetuned_experiment-1_checkpoint-3000_fleurs-test.jsonl", "w") as f:
    for transcript in transcriptions_val:
        json.dump(transcript, f)
        f.write("\n")

In [ ]:
transcriptions = [elem[0] for elem in transcriptions_val]

In [ ]:
transcriptions

In [ ]:
cache_dir="/data/users/kashrest/asr-experiments"
data = load_dataset("google/fleurs", "ha_ng", split="test", cache_dir=cache_dir)
labels = []
for elem in (iter(data)):
    labels.append(elem["transcription"])

In [ ]:
print(labels[54])

In [ ]:
import json
transcriptions = []
with open("whisper-tiny-zero-shot_hausa_fleurs-test.jsonl", "r") as f:
    for line in f:
        transcriptions.extend(json.loads(line))

In [ ]:
#for i, elem in enumerate(transcriptions):
    #print(i, elem)
    
transcriptions[54]

In [ ]:
from datasets import load_metric

labels = []
for elem in iter(data):
    labels.append(elem["transcription"])

wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

wer_metric.compute(predictions=transcriptions, references=labels)

In [ ]:
cer_metric.compute(predictions=transcriptions, references=labels)

In [ ]:
labels

In [ ]:
n = 102
print(f"Predicted transcript:\n{transcriptions[n]}\n\nOriginal:\n{labels[n]}")
print("\n****\n")